In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
from IPython.core.debugger import set_trace

In [ ]:
CAMERA_IDS = [
    'FrontSS', 'LeftSS', 'RightSS', 'RearSS', 'TopSS'
]

CLASSES_NAMES = [
    ['Roads', 'RoadLines'],
    
    ['None', 'Buildings', 'Fences', 'Other', 'Pedestrians',
     'Poles', 'Walls', 'TrafficSigns',
     'Vegetation', 'Sidewalks'],
    
    ['Vehicles'],
]

CLASSES_NAMES = [
    ['Roads', 'RoadLines'],
    
    ['Sidewalks'],
    
    ['Buildings'],
    
    ['Fences', 'Other', 'Pedestrians',
     'Poles', 'Walls', 'TrafficSigns'],
    
    ['Vehicles'],
    
    ['Vegetation'],
        
    ['None']
]


DECIMATION = 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from utils import (
    get_X_and_Y, make_movie, unwrap_to_ohe, class_names_to_class_numbers
)


storage = get_X_and_Y(['Town01'], range(1), DECIMATION, CAMERA_IDS)
X = [storage[id_] for id_ in CAMERA_IDS]

In [ ]:
frame = 100
camera_idx = 0


fig, ax = plt.subplots()
ax.grid(False)
fig.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0, hspace=0)
fig.set_size_inches((20, 10))
ax.margins(0, 0)

for frame in range(0, 1000, 100):
    for camera_idx in range(5):
        x = unwrap_to_ohe(X[camera_idx][..., frame], class_names_to_class_numbers(CLASSES_NAMES))
        ax.imshow(np.argmax(x, axis=2), vmin=0, vmax=len(CLASSES_NAMES), cmap='gist_stern')#, aspect='auto')
        ax.axis('off')
        fig.show()
        plt.savefig('images/for_blog_post_frame={}_camera={}.png'.format(frame, camera_idx))

In [ ]:
multi_model = None

for racetrack in ['Town01', 'Town02']:
    for episode in [102, 103]:
        multi_model = make_movie(
            'models/multi_model__sweep=21_decimation=2_numclasses=7_valloss=0.305.h5',
            racetrack,
            episode,
            DECIMATION,
            CLASSES_NAMES,
            CAMERA_IDS
        )

In [ ]:
frame = 10


plt.imshow(X[0][..., 0, frame], vmin=0, cmap='gist_stern')

In [ ]:
from sklearn.metrics import log_loss, roc_auc_score


episode_len = 1000


X, Y = get_X_and_Y(['Town01', 'Town02'], [0, 1, 2, 3, 4, 5, 6], DECIMATION)
    
classes_numbers = class_names_to_class_numbers(CLASSES_NAMES)

X_final, y_final = [[] for _ in range(4)], []
for index in range(episode_len):
    X_out, y_out = extract_observation_for_batch(X, Y, index, False, classes_numbers)
    for j in range(len(X_final)):
        X_final[j].append(X_out[j])
    y_final.append(y_out)

X_final = [np.stack(x) for x in X_final]
y_final = np.stack(y_final)

preds = multi_model.predict(X_final + [y_final])[5]

In [ ]:
loss_matrix = np.zeros(y_final.shape[1:3])

# for frame in range(loss_matrix.shape[0]):
for i in range(loss_matrix.shape[0]):
    for j in range(loss_matrix.shape[1]):
        loss_matrix[i, j] = log_loss(y_final[:, i, j], preds[:, i, j])

In [ ]:
plt.figure(figsize=(40, 10))

plt.imshow(loss_matrix.T, cmap='seismic')
plt.colorbar()